In [ ]:
%matplotlib ipympl

import matplotlib.pyplot as plt
import numpy as np
import pykonal
import scipy.stats
import seispy

EARTH_RADIUS = 6371.

MEAN_P_RESIDUAL = 0.
STD_P_RESIDUAL = 0.15
MEAN_S_RESIDUAL = 0.0
STD_S_RESIDUAL = 0.25

In [ ]:
db = seispy.pandas.catalog.Catalog(
    "/home/malcolmw/google_drive/malcolm.white@usc.edu/data/events/malcolmw/SJFZ_catalog_2008-2016.h5",
    fmt="hdf5"
)

In [ ]:
df_event = db["origin"].merge(
    db["origerr"][["orid", "smajax", "sdepth"]],
    on="orid"
)
df_event = df_event[
     (df_event["algorithm"] == "GrowClust")
    |(
         (df_event["algorithm"] == "NonLinLoc")
        &(df_event["smajax"] < 3)
        &(df_event["sdepth"] < 3)
    )
]

In [ ]:
df = db["assoc"][["arid", "orid", "sta", "phase"]][
    db["assoc"]["orid"].isin(df_event["orid"])
].merge(
    df_event[["orid", "evid"]],
    on="orid"
).merge(
    db["arrival"][["arid", "time"]],
    on="arid"
).merge(
    db["snetsta"][["snet", "sta"]].drop_duplicates("sta"),
    on="sta"
)
df["station_id"] = df["snet"] + "." + df["sta"]
df = df.drop(
    columns=["arid", "orid", "snet", "sta"]
).sort_values(
    ["evid", "time"]
).set_index("evid")

In [ ]:
def geo2sph(geo):
    geo = np.array(geo, dtype=np.float64)
    sph = np.empty_like(geo)
    sph[...,0] = EARTH_RADIUS - geo[...,2]
    sph[...,1] = np.radians(90 - geo[...,0])
    sph[...,2] = np.radians(geo[...,1])
    return (sph)

def sph2geo(sph):
    sph = np.array(sph, dtype=np.float64)
    geo = np.empty_like(sph)
    geo[...,0] = np.degrees(np.pi/2 - sph[...,1])
    geo[...,1] = np.degrees(sph[...,2])
    geo[...,2] = EARTH_RADIUS - sph[...,0]
    return (geo)


def likelihood(model, data, priors, interpolators):
    """
    Return the likelihood of the model given data and priors.
    
    :param model: lat, lon, depth, time coordinates of hypocenter.
    :type model: Array-like of floats.
    :param data: Station ID, phase, arrival time triplets for observed data.
    :type data: tuple(str, str, float)
    :param priors: Empirical residual PDFs in dict with (station ID, phase) keys. Values should be scipy.stats.rv_continuous.pdf or similar.
    :type priors: dict
    :param network: Station locations in dict where key is station ID and values are tuples with lat, lon, depth coordinates.
    """
    data["key"] = list(zip(data["station_id"], data["phase"]))
    data = data[data["key"].isin(interpolators)]
    return (
        np.prod([
            priors[row["phase"]](row["time"]-(model[3] + interpolators[(row["station_id"], row["phase"])](geo2sph(model[:3]))))
            for idx, row in data.iterrows()
        ])
    )


def load_interpolator_from_file(filename):
    try:
        npz = np.load(filename)
        grid = pykonal.Grid3D(coord_sys="spherical")
        grid.min_coords = npz["min_coords"]
        grid.node_intervals = npz["node_intervals"]
        grid.npts = npz["npts"]
        field = npz["uu"]
    finally:
        npz.close()
    return (pykonal.LinearInterpolator3D(grid, field))

In [ ]:
tti = load_interpolator_from_file("/home/malcolmw/scratch/traveltimes_HK1D/AZ.CRY.P.npz")
grid = pykonal.Grid3D(coord_sys="spherical")
grid.min_coords = tti.min_coords
grid.node_intervals = tti.node_intervals
grid.npts = tti.npts

In [ ]:
priors = dict(
    P=scipy.stats.norm(MEAN_P_RESIDUAL, STD_P_RESIDUAL).pdf,
    S=scipy.stats.norm(MEAN_S_RESIDUAL, STD_S_RESIDUAL).pdf
)

Let $\mathbf{m}$ and $\mathbf{d}$ be vectors comprising a set of model parameters and observed data, respectively. Bayes theorem states

\begin{equation}
    P\left(\mathbf{m}\;\middle|\;\mathbf{d}\right) = \frac{
        P\left(\mathbf{d}\;\middle|\;\mathbf{m}\right)P\left(\mathbf{m}\right)
    }{
        P\left(\mathbf{d}\right)
    }.
\end{equation}

In the case of locating earthquakes, $\mathbf{m}=\mathbf{h_0}=\left[\rho_0, \theta_0, \phi_0, t_0\right]$ is the vector of hypocenter parameters and $\mathbf{d}$ is a vector of observed phase-arrival times. The left hand side is known as the *posterior* probability, and represents the probability that any particular model underlies the oberved data. Locating an earthquake is tantamount to determining the model that most probably underlies the observed data.

The denominator of the right hand side is known as the *evidence*. Because the evidence for an earthquake is constant and we are only interested in the relative maximum of the left hand side, we can safely ignore it as it plays the role of a scaling constant. 

The first term of numerator on the right hand side, $P\left(\textbf{d}\;\middle|\;\textbf{m}\right)$, represents our knowledge of how the data are related to the model---i.e., for a given model, what is the probability of observing the data that was observed---and is known as the *prior* probability. 

Finally, the second term of the numerator on the right hand side, $P\left(\textbf{m}\right)$ is the probability of the model taking on a particular set of parameters. A priori knowledge of the distribution of model parameters can be encoded here, or one can plead ignorance and simply assign all possible combinations of parameters equal probability, which is equivalent to setting $P\left(\textbf{m}\right)$ equal to a uniform distribution.

In the end, we have
\begin{equation}
    P\left(\mathbf{h}_0\;\middle|\;\mathbf{d}\right) \propto P\left(\mathbf{d}\;\middle|\;\mathbf{h}_0\right)
\end{equation}

and we are interested in
\begin{equation}
    \underset{\mathbf{h}_0}{argmax} \left[P\left(\mathbf{h}_0\;\middle|\;\mathbf{d}\right)\right]
\end{equation}

In [ ]:
interpolators = dict()
tt = []
data = df.loc[8000025]
for idx, row in data.iterrows():
    station_id, phase = row[["station_id", "phase"]]
    try:
        interpolators[(station_id, phase)] = load_interpolator_from_file(
            f"/home/malcolmw/scratch/traveltimes_HK1D/{station_id}.{phase}.npz"
        )
        tt.append(row["time"] - interpolators[(f"{station_id}", phase)].values)
    except:
        continue
tt = np.stack(tt)
idx_min = np.unravel_index(np.argmin(tt.std(axis=0)), tt.shape[1:])
hypo0 = np.append(sph2geo(grid[idx_min]), tt.mean(axis=0)[idx_min]) # Initial hypocenter estimate
likelihood(hypo0, data, priors, interpolators)

In [ ]:
lat_max, lon_min, depth_max = sph2geo(grid.min_coords)
lat_min, lon_max, depth_min = sph2geo(grid.max_coords)

In [ ]:
%%time
sd_step_lat = np.degrees(grid.node_intervals[1]) / 20
sd_step_lon = np.degrees(grid.node_intervals[2]) / 20
sd_step_depth = grid.node_intervals[0] / 20
sd_step_time = 0.05
nstep = 500
ntraces = 4

lat_max, lon_min, depth_max = sph2geo(grid.min_coords)
lat_min, lon_max, depth_min = sph2geo(grid.max_coords)

traces = []
map_hypos = []
for itrace in range(ntraces):
    while True:
        hypo_prev = hypo0 + np.random.randn() * np.array([sd_step_lat, sd_step_lon, sd_step_depth, sd_step_time])
        likeli_prev = likelihood(hypo_prev, data, priors, interpolators)
        max_likeli, map_hypo = likeli_prev, hypo_prev
        if likeli_prev > 0:
            break
    trace = np.array([])

    for istep in range(nstep):
        hypo_prop = np.empty(4)
        while True:
            lat_next = hypo_prev[0] + np.random.randn() * sd_step_lat
            if lat_min <= lat_next <= lat_max:
                hypo_prop[0] = lat_next
                break
        while True:
            lon_next = hypo_prev[1] + np.random.randn() * sd_step_lon
            if lon_min <= lon_next <= lon_max:
                hypo_prop[1] = lon_next
                break
        while True:
            depth_next = hypo_prev[2] + np.random.randn() * sd_step_depth
            if depth_min <= depth_next <= depth_max:
                hypo_prop[2] = depth_next
                break
        hypo_prop[3] = hypo_prev[3] + np.random.randn() * sd_step_time
        likeli_prop = likelihood(hypo_prop, data, priors, interpolators)
        likeli_prev = likelihood(hypo_prev, data, priors, interpolators)
        rv = np.random.rand()
        alpha = likeli_prop/likeli_prev
        if alpha >= rv:
            trace = np.append(trace, hypo_prop)
            hypo_prev = hypo_prop
            likeli_prev = likeli_prop
            if likeli_prop > max_likeli:
                max_likeli = likeli_prop
                map_hypo = hypo_prop
        else:
            trace = np.append(trace, hypo_prev)
    trace = trace.reshape(nstep, 4)
    traces.append(trace)
    map_hypos.append(map_hypo)

In [ ]:
plt.close("all")
fig = plt.figure()
ax0 = fig.add_subplot(2, 2, 1)
ax1 = fig.add_subplot(2, 2, 2)
ax2 = fig.add_subplot(2, 2, 3)
ax3 = fig.add_subplot(2, 2, 4)
for itrace in range(ntraces):
    trace = traces[itrace]
    map_hypo = map_hypos[itrace]
#     counts, bins, patches = ax0.hist(trace[:,0], bins=50, density=True)
    counts, bins = np.histogram(trace[:,0], bins=50, density=True)
    kde = scipy.stats.gaussian_kde(trace[:,0])
    ax0.plot(bins, kde(bins))
    ax0.scatter(map_hypo[0], kde(map_hypo[0]), zorder=3)

#     counts, bins, patches = ax1.hist(trace[:,1], bins=50, density=True)
    counts, bins = np.histogram(trace[:,1], bins=50, density=True)
    kde = scipy.stats.gaussian_kde(trace[:,1])
    ax1.plot(bins, kde(bins))
    ax1.scatter(map_hypo[1], kde(map_hypo[1]), zorder=3)

#     counts, bins, patches = ax2.hist(trace[:,2], bins=50, density=True)
    counts, bins = np.histogram(trace[:,2], bins=50, density=True)
    kde = scipy.stats.gaussian_kde(trace[:,2])
    ax2.plot(bins, kde(bins))
    ax2.scatter(map_hypo[2], kde(map_hypo[2]), zorder=3)

#     counts, bins, patches = ax3.hist(trace[:,3], bins=50, density=True)
    counts, bins = np.histogram(trace[:,3], bins=50, density=True)
    kde = scipy.stats.gaussian_kde(trace[:,3])
    ax3.plot(bins, kde(bins))
    ax3.scatter(map_hypo[3], kde(map_hypo[3]), zorder=3)

In [ ]:
fig = plt.figure()
for itrace in range(len(traces)):
    map_hypo = map_hypos[itrace]
    trace = traces[itrace]
    ax = fig.add_subplot(2, 2, itrace+1)
    bm = seispy.mapping.Basemap(
    #     continent_color="0.75",
        ax=ax
    )
    bm.scatter(trace[:,1], trace[:,0], s=1, linewidth=0, c="k", zorder=2)
#     bm.scatter(map_hypo[1], map_hypo[0], c="r", zorder=2)

In [ ]:
traces